In [123]:
import pandas as pd
import numpy as np
import os


### Read the monthly trade data from Jan 2000 to Sep 2023

The time range is that the trade data for most countries are updated till Sep 2023

In [124]:
# Specify the path to the folder containing CSV files
folder_path = '../data/raw/MonthlyTradeData'

# Get a list of all files in the folder
files = [file for file in os.listdir(folder_path)]

# Create an empty list to store DataFrames
dfs = []

# Loop through each CSV file and read it into a DataFrame
for file in files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path, encoding='unicode_escape')
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame if needed
monthly_TradeData = pd.concat(dfs)

monthly_TradeData = monthly_TradeData[(monthly_TradeData['Period']<=202309)]


# Keep the necessary columns
monthly_TradeData = monthly_TradeData[['Period','ReporterISO','ReporterDesc','PartnerISO','PartnerDesc','PrimaryValue']]


# Save to csv
monthly_TradeData.to_csv('../data/processed/monthly_TradeData.csv',index=False)

In [130]:
# Pre-processing data

# Define a function to strip whitespace for categorical values
def strip_categorical(value):
    if isinstance(value, str):
        return value.strip()
    return value

def replace_and_delete_countries(df):
    # Apply the function to the entire DataFrame
    df = df.applymap(strip_categorical)
    
    # Replace "Other Asia, nes" (S19) to Taiwan,China (TWM)
    df['ReporterISO'] = df['ReporterISO'].replace('S19', 'TWN')
    df.loc[df['ReporterISO'] == 'TWN', 'ReporterDesc'] = 'China, Taiwan'
    
    df['PartnerISO'] = df['PartnerISO'].replace('S19', 'TWN')
    df.loc[df['PartnerISO'] == 'TWN', 'PartnerDesc'] = 'China, Taiwan'
    
    # Delete other desc containing ', nes'
    # Replace values containing ', nes' with NaN
    df.loc[df['ReporterDesc'].str.contains(', nes', case=False, na=False), 'ReporterDesc'] = np.nan
    df.loc[df['PartnerDesc'].str.contains(', nes', case=False, na=False), 'PartnerDesc'] = np.nan
    # Delete rows containing Nan
    df = df.dropna()
    
    # Delete 'Free Zone', 'Bunkers', 'Special Categories', 'World', 'Neutral zone' 
    to_delete = ['Free Zone','Free Zones','Bunkers','Special Categories','World','Neutral Zone']
    
    # Delete rows where source is in the list of countries
    df = df[~df['ReporterDesc'].isin(to_delete)]
    # Delete rows where target is in the list of countries
    df = df[~df['PartnerDesc'].isin(to_delete)]

    return df


In [131]:
monthly_TradeData = replace_and_delete_countries(monthly_TradeData)
monthly_TradeData.info()

# Save to csv
monthly_TradeData.to_csv('../data/processed/monthly_TradeData.csv',index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 545537 entries, 0 to 37901
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Period        545537 non-null  int64  
 1   ReporterISO   545537 non-null  object 
 2   ReporterDesc  545537 non-null  object 
 3   PartnerISO    545537 non-null  object 
 4   PartnerDesc   545537 non-null  object 
 5   PrimaryValue  545537 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 29.1+ MB


### Get the country list

In [132]:
# Creaet Appendix for country name and ISO code
def get_country_list(df):
    df = df[['ReporterISO','ReporterDesc','PartnerISO','PartnerDesc']]

    # Obtain the countries in the links
    countries_code = pd.concat([df[['ReporterISO']].rename(columns={'ReporterISO': 'ISO'}),df[['PartnerISO']].rename(columns={'PartnerISO': 'ISO'})]).drop_duplicates()
    
    # Match company info with selected companies
    countries = pd.merge(countries_code,df,left_on='ISO', right_on='ReporterISO', how='left')
    countries = countries[['ISO','ReporterDesc']].rename(columns={'ReporterDesc':'Name'}).drop_duplicates()
    
    countries = pd.merge(countries, df, left_on='ISO',right_on='PartnerISO', how='left')
    countries = countries[['ISO','PartnerDesc']].rename(columns={'PartnerDesc':'Name'}).drop_duplicates()
    
    return countries



In [167]:
countries = get_country_list(monthly_TradeData)
countries.to_csv('../data/processed/countries.csv',index=False, encoding='utf')

In [134]:
countries.info()


<class 'pandas.core.frame.DataFrame'>
Index: 238 entries, 0 to 545525
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISO     238 non-null    object
 1   Name    238 non-null    object
dtypes: object(2)
memory usage: 5.6+ KB


### Read EPU DATA

In [162]:
# Define the range of rows you want to read (37:321)
epu = pd.read_excel('../data/raw/Global_Policy_Uncertainty_Data.xlsx').iloc[36:321]

# Assuming df is your DataFrame
epu['period'] = epu['Year'].astype(str) + 'M' + epu['Month'].astype(int).apply(lambda x: f'{x:02d}')

# Select columns
epu = epu[["period", "GEPU_current","GEPU_ppp"]]

epu.to_csv('../data/processed/epu.csv', index=False, encoding='utf')

In [163]:
epu

,period,GEPU_current,GEPU_ppp
36,2000M01,63.738693,61.930168
37,2000M02,58.820887,53.231096
38,2000M03,61.289983,58.165654
39,2000M04,64.557214,61.171080
40,2000M05,87.164595,84.551517
...,...,...,...
316,2023M05,228.814279,227.546243
317,2023M06,231.438806,240.770257
318,2023M07,228.340938,238.605431
319,2023M08,213.460434,226.492244
